In [ ]:
import os
import numpy as np
import cv2
import dlib
import kagglehub
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

# Step 1: Download the dataset
print("Downloading iBUG 300-W dataset...")
path = kagglehub.dataset_download("toxicloser/ibug-300w-large-face-landmark-dataset")
print(f"Dataset downloaded to: {path}")

# Step 2: Set up dlib face detector and landmark predictor
print("Setting up dlib models...")
detector = dlib.get_frontal_face_detector()
predictor_path = "shape_predictor_68_face_landmarks.dat"  # Make sure you have this file

# Check if predictor file exists, if not download it
if not os.path.exists(predictor_path):
    print("Downloading dlib's 68 point facial landmark predictor...")
    # You can download it from dlib's GitHub or website
    # For this example, please download it manually from:
    # http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
    print(f"Please download the predictor file from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
    print(f"Extract it and place it in the current directory as {predictor_path}")
    exit(1)

predictor = dlib.shape_predictor(predictor_path)

# Function to convert dlib shape to numpy array
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

# Function to read the iBUG landmark format
def read_landmarks(landmark_file):
    with open(landmark_file, 'r') as f:
        lines = f.readlines()
    
    # Skip the header and version info
    lines = [line.strip() for line in lines if not line.startswith('version') and line.strip()]
    
    # Extract number of points
    n_points = int(lines[0].split()[1])
    
    # Read landmark coordinates
    landmarks = np.zeros((n_points, 2))
    for i in range(n_points):
        x, y = map(float, lines[i+1].split())
        landmarks[i] = [x, y]
    
    return landmarks

# Function to calculate the normalized mean error (NME)
def calculate_nme(pred_landmarks, gt_landmarks, norm_factor):
    """
    Calculate the Normalized Mean Error
    norm_factor: usually the inter-ocular distance (distance between eyes)
    """
    assert pred_landmarks.shape == gt_landmarks.shape, "Shapes don't match"
    
    # Calculate Euclidean distance for each landmark
    error = np.sqrt(np.sum((pred_landmarks - gt_landmarks)**2, axis=1))
    
    # Normalize by the normalization factor
    normalized_error = error / norm_factor
    
    # Return mean normalized error
    return np.mean(normalized_error)

# Function to calculate inter-ocular distance (distance between eyes)
def calculate_inter_ocular_distance(landmarks):
    # For 68-point landmarks, points 36-41 are right eye, 42-47 are left eye
    right_eye_center = np.mean(landmarks[36:42], axis=0)
    left_eye_center = np.mean(landmarks[42:48], axis=0)
    return np.linalg.norm(right_eye_center - left_eye_center)

# Main evaluation function
def evaluate_dlib_accuracy():
    # Find all images and their corresponding landmark files
    dataset_dir = path
    subsets = ['ibug', 'afw', 'helen/trainset', 'helen/testset', 'lfpw/trainset', 'lfpw/testset']
    
    all_errors = []
    subset_errors = {subset: [] for subset in subsets}
    
    total_images = 0
    processed_images = 0
    failed_detections = 0
    
    for subset in subsets:
        subset_path = os.path.join(dataset_dir, subset)
        if not os.path.exists(subset_path):
            print(f"Warning: Path {subset_path} does not exist. Skipping.")
            continue
            
        image_files = glob.glob(os.path.join(subset_path, '*.jpg')) + glob.glob(os.path.join(subset_path, '*.png'))
        total_images += len(image_files)
        
        print(f"\nProcessing {subset}: {len(image_files)} images")
        
        for img_path in tqdm(image_files):
            # Find corresponding landmark file
            pts_path = os.path.splitext(img_path)[0] + '.pts'
            if not os.path.exists(pts_path):
                print(f"Warning: No landmark file for {img_path}. Skipping.")
                continue
                
            # Read image
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Could not read image {img_path}. Skipping.")
                continue
                
            # Read ground truth landmarks
            gt_landmarks = read_landmarks(pts_path)
            
            # Run dlib detector
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 1)
            
            if len(rects) == 0:
                print(f"Warning: No face detected in {img_path}. Skipping.")
                failed_detections += 1
                continue
                
            # Use the first detected face
            rect = rects[0]
            
            # Apply landmark predictor
            shape = predictor(gray, rect)
            pred_landmarks = shape_to_np(shape)
            
            # Calculate normalization factor (inter-ocular distance)
            iod = calculate_inter_ocular_distance(gt_landmarks)
            
            # Calculate error
            error = calculate_nme(pred_landmarks, gt_landmarks, iod)
            all_errors.append(error)
            subset_errors[subset].append(error)
            
            processed_images += 1
    
    # Print overall statistics
    print("\n--- Evaluation Results ---")
    print(f"Total images: {total_images}")
    print(f"Processed images: {processed_images}")
    print(f"Failed detections: {failed_detections}")
    print(f"Detection rate: {processed_images/total_images*100:.2f}%")
    
    if processed_images > 0:
        print(f"\nOverall Mean NME: {np.mean(all_errors)*100:.4f}%")
        print(f"Overall Median NME: {np.median(all_errors)*100:.4f}%")
        
        # Print per-subset statistics
        print("\nPer-subset NME:")
        for subset in subsets:
            if subset_errors[subset]:
                print(f"{subset}: {np.mean(subset_errors[subset])*100:.4f}%")
        
        # Plot error distribution
        plt.figure(figsize=(10, 6))
        plt.hist(np.array(all_errors)*100, bins=50, alpha=0.7)
        plt.xlabel('Normalized Mean Error (%)')
        plt.ylabel('Frequency')
        plt.title('Distribution of Landmark Detection Errors')
        plt.axvline(np.mean(all_errors)*100, color='r', linestyle='dashed', linewidth=1, label=f'Mean: {np.mean(all_errors)*100:.4f}%')
        plt.axvline(np.median(all_errors)*100, color='g', linestyle='dashed', linewidth=1, label=f'Median: {np.median(all_errors)*100:.4f}%')
        plt.legend()
        plt.savefig('dlib_error_distribution.png')
        print("Error distribution plot saved as 'dlib_error_distribution.png'")
        
        # Plot some example images with landmarks
        plot_sample_results(dataset_dir, subsets)

def plot_sample_results(dataset_dir, subsets, num_samples=5):
    """Plot some sample results to visualize the landmark detection"""
    plt.figure(figsize=(15, 3*num_samples))
    
    sample_count = 0
    for subset in subsets:
        subset_path = os.path.join(dataset_dir, subset)
        if not os.path.exists(subset_path):
            continue
            
        image_files = glob.glob(os.path.join(subset_path, '*.jpg')) + glob.glob(os.path.join(subset_path, '*.png'))
        if not image_files:
            continue
            
        # Take a random sample
        import random
        sample_files = random.sample(image_files, min(num_samples//len(subsets) + 1, len(image_files)))
        
        for img_path in sample_files:
            if sample_count >= num_samples:
                break
                
            # Find corresponding landmark file
            pts_path = os.path.splitext(img_path)[0] + '.pts'
            if not os.path.exists(pts_path):
                continue
                
            # Read image
            img = cv2.imread(img_path)
            if img is None:
                continue
                
            # Read ground truth landmarks
            gt_landmarks = read_landmarks(pts_path)
            
            # Run dlib detector
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 1)
            
            if len(rects) == 0:
                continue
                
            # Use the first detected face
            rect = rects[0]
            
            # Apply landmark predictor
            shape = predictor(gray, rect)
            pred_landmarks = shape_to_np(shape)
            
            # Calculate normalization factor (inter-ocular distance)
            iod = calculate_inter_ocular_distance(gt_landmarks)
            
            # Calculate error
            error = calculate_nme(pred_landmarks, gt_landmarks, iod)
            
            # Plot
            plt.subplot(num_samples, 3, sample_count*3 + 1)
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(f'Original Image\n{os.path.basename(img_path)}')
            plt.axis('off')
            
            plt.subplot(num_samples, 3, sample_count*3 + 2)
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            for i in range(68):
                plt.plot(gt_landmarks[i, 0], gt_landmarks[i, 1], 'go', markersize=3)
            plt.title('Ground Truth Landmarks')
            plt.axis('off')
            
            plt.subplot(num_samples, 3, sample_count*3 + 3)
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            for i in range(68):
                plt.plot(pred_landmarks[i, 0], pred_landmarks[i, 1], 'ro', markersize=3)
            plt.title(f'Dlib Predictions\nNME: {error*100:.2f}%')
            plt.axis('off')
            
            sample_count += 1
            if sample_count >= num_samples:
                break
    
    plt.tight_layout()
    plt.savefig('landmark_detection_samples.png')
    print("Sample landmark detection results saved as 'landmark_detection_samples.png'")



In [ ]:
evaluate_dlib_accuracy()